<h2><center>Procedure for Estimation and Reporting of Uncertainty Due to Discretization
in CFD Applications</center></h2>

<h4><center>Aerodynamic evaluation of modified SG6043 airfoils using
the γ − Reθ turbulence model with OpenFOAM</center></h4>

This notebook implements the procedure for uncertainty quantification due to meshing in a CFD application according to the Fluids Engineering Division of ASME. The recommended procedure for the estimation of discretization error is as follows [1]:

Step 1: Define a representative mesh / cell / grid size $h$,
$$
    h = \left[ \frac{1}{N}\sum_{i=1}^{N} (\Delta V_i) \right]^{1/3}\quad\mathrm{3D}
$$

$$
    h = \left[ \frac{1}{N}\sum_{i=1}^{N} (\Delta A_i) \right]^{1/2}\quad\mathrm{2D}
$$


Step 2: Select 3 significantly different sets of grids and run simulations to determine the values of key variables important to the objective of the simulation study. It is prefereble that the grid refinement factor satisfies the following condition:

$$
r = h_{\mathrm{coarse}}/h_{\mathrm{fine}} > 1.3
$$

Step 3: Let $h_1 < h_2 < h_3$, $r_{21} = h_2/h_1$, $r_{32} = h_3/h_2$, and $\varepsilon_{32}=\phi_3-\phi_2$, $\varepsilon_{21}=\phi_2-\phi_1$ calculate the apparent order p of the method using, 

$$
p = \frac{1}{\ln{(r_{21})}}|\ln|\varepsilon_{32}/\varepsilon_{21}|+q(p)|
$$

with, 

$$
q(p) = \ln\left(\frac{r_{21}^p-s}{r_{32}^p-s}\right)
$$

and 

$$
s = 1·\mathrm{sgn}(\varepsilon_{32}/\varepsilon_{21})
$$




Step 4: Calculate the extrapolated values from, 
$$
\phi_{\mathrm{ext}}^{21} = \frac{(r_{21}^p\phi_1-\phi_2)}{(r_{21}^p-1)} 
$$


Step 5: Calculate and report the following error estimates with the apparent order $p$, 

$$
e_a^{21} = \left| \frac{\phi_1-\phi_2}{\phi_1}\right|
$$

$$
e_{\mathrm{ext}}^{21} = \left| \frac{\phi_{\mathrm{ext}}^{21}-\phi_1}{\phi_{\mathrm{ext}}^{21}}\right|
$$

$$
GCI_{\mathrm{fine}}^{21} = \frac{1.25e_{a}^{21}}{r_{21}^p-1}
$$

In [1]:
# Array handling and numerical methods
import numpy as np
from scipy.optimize import fixed_point

# plotting and visualizing
import matplotlib.pyplot as plt

# OpenFOAM file handling
import os
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile

In [61]:
def get_meshsweep_data(sweepdir):

    coefs = []
    h = []

    for file in os.listdir(sweepdir):
        
        volumeFile = ParsedParameterFile(os.path.join(sweepdir, file, "5000/V"))
        volume = np.array(volumeFile["internalField"])
        coeffFile = np.loadtxt(os.path.join(sweepdir, file,  "postProcessing/forceCoeffs/0/forceCoeffs.dat"))

        coefs.append(np.mean(coeffFile, axis=0)[2:4])
        h.append(np.mean(volume)**(1/3))
        
    coefs = np.array(coefs)
    h = np.array(h)
        
    return coefs, h

def q(p, r, s):
    return np.log((r[0]**p-s)/(r[1]**p-s))
                                   
def p(p, eps, r):
    feps = eps[1]/eps[0]
    resp = (1/np.log(r[0]))*np.abs(np.log(np.abs(feps))+q(p, r, np.sign(feps)))
                                      
    return resp
def uncertainty_quant(sweepdir):
    """
        Calculates the grid convergence index of the simulation 
        based on the mesh convergence study.
    """

    coefs, h = get_meshsweep_data(sweepdir)
    eps = np.diff(coefs, axis=0)
    
    r = h[:-1]/h[1:]
    r = r.reshape(-1, 1)
    
    feps = eps[1:]/eps[:-1]
    
    p0 = (1/np.log(r[1:]))*np.abs(np.log(np.abs(feps)))
    px = np.zeros_like(p0)
    
    for i in range(len(p0)):
        
        R0 = [r[i], r[i+1]]
        R1 = [r[i], r[i+1]]
        
        EPS0 = [eps[i,0], eps[i+1,0]]
        EPS1 = [eps[i,1], eps[i+1,1]]
        
        px[i, 0] = fixed_point(p, p0[i,0], (EPS0, R0))
        px[i, 1] = fixed_point(p, p0[i,1], (EPS1, R1))
    
    return px

In [62]:
uncertainty_quant("../meshsweep_NREL/")

/tmp/ipykernel_54456/3260073427.py:21: RuntimeWarning: overflow encountered in power
  return np.log((r[0]**p-s)/(r[1]**p-s))


RuntimeError: Failed to converge after 500 iterations, value is [nan]

[1]“Procedure for Estimation and Reporting of Uncertainty Due to Discretization in CFD Applications” 
Journal of Fluids Engineering, vol. 130, no. 7. ASME International, p. 078001, 2008. doi: 10.1115/1.2960953.